In [1]:
!pip install torch transformers datasets peft bitsandbytes accelerate pandas pillow huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [2]:
    from huggingface_hub import notebook_login

    notebook_login()

In [ ]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
import requests
import torch

model_id = "google/paligemma-3b-mix-224"

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

model = PaliGemmaForConditionalGeneration.from_pretrained(model_id).eval()
processor = AutoProcessor.from_pretrained(model_id)

# Instruct the model to create a caption in Spanish
prompt = "caption es"
model_inputs = processor(text=prompt, images=image, return_tensors="pt")
input_len = model_inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]
    decoded = processor.decode(generation, skip_special_tokens=True)
    print(decoded)


In [4]:
import os
file_path = "/usr/local/lib/python3.11/dist-packages/transformers/models/paligemma/processing_paligemma.py"
with open(file_path, 'r') as f:
    content = f.read()
if 'import numpy as np' not in content:
    with open(file_path, 'w') as f:
        f.write("import numpy as np\n" + content)

In [5]:
# Full script for fine-tuning PaliGemma (Google's VLM based on Gemma 2B) on a custom Parquet dataset for image captioning
# Adapted from Hugging Face blog: https://huggingface.co/blog/paligemma
# Requirements: Run on a machine with GPU, install dependencies via:
# pip install torch transformers datasets peft bitsandbytes accelerate pandas pillow huggingface_hub
# Login to Hugging Face if pushing model: huggingface-cli login

import torch
from torch.utils.data import Dataset
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset as HFDataset
import pandas as pd
import numpy as np
from io import BytesIO
from PIL import Image
from huggingface_hub import notebook_login  # Or use CLI login

# Optional: Login to Hugging Face
# notebook_login()

# Custom Dataset for Parquet file
class ParquetCaptionDataset(Dataset):
    def __init__(self, parquet_path):
        self.df = pd.read_parquet(parquet_path)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(BytesIO(row['image'])).convert("RGB")
        caption = row['caption']
        return {
            "image": image,
            "caption": caption,
            # If you have questions, add here; for pure captioning, we'll use a fixed prefix
        }

# Load your dataset (assume full for train; split if needed)
parquet_path = "/kaggle/input/appron-prompt-dataset/dataset.parquet"
custom_dataset = ParquetCaptionDataset(parquet_path)
hf_dataset = HFDataset.from_list([custom_dataset[i] for i in range(len(custom_dataset))])  # Convert to HF Dataset
hf_dataset = hf_dataset.train_test_split(test_size=0.1)  # Split into train/val
train_ds = hf_dataset["train"]
val_ds = hf_dataset["test"]

# Load processor
model_id = "google/paligemma-3b-pt-224"  # PaliGemma 3B with pre-trained resolution 224
processor = PaliGemmaProcessor.from_pretrained(model_id)

# Collate function for batching (adapted for captioning)
image_token = processor.tokenizer.convert_tokens_to_ids("<image>")
def collate_fn(examples):
    texts = ["caption en\n" for example in examples]  # Fixed prefix for English captioning; adjust if needed (e.g., "describe this image")
    labels = [example['caption'] for example in examples]
    images = [example["image"] for example in examples]
    tokens = processor(text=texts, images=images, suffix=labels,
                       return_tensors="pt", padding="longest",
                       tokenize_newline_separately=False)  # Important for PaliGemma

    tokens = tokens.to(torch.bfloat16)
    return tokens

# Load model with quantization for efficiency
device = "cuda" if torch.cuda.is_available() else "cpu"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"  # Auto-maps to GPU
)

# Freeze vision tower and projector if desired (optional, but recommended for efficiency)
for param in model.vision_tower.parameters():
    param.requires_grad = False
for param in model.multi_modal_projector.parameters():
    param.requires_grad = False  # Or True if you want to fine-tune projector

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Check trainable params

# Training arguments (adjust based on your hardware/dataset size)
args = TrainingArguments(
    num_train_epochs=2,  # Start small
    remove_unused_columns=False,
    per_device_train_batch_size=4,  # Adjust based on VRAM (with 4bit, should fit)
    gradient_accumulation_steps=4,
    warmup_steps=2,
    learning_rate=2e-5,
    weight_decay=1e-6,
    adam_beta2=0.999,
    logging_steps=100,
    optim="adamw_torch",
    save_strategy="epoch",
    push_to_hub=False,  # Set to True to push to HF
    output_dir="finetuned_paligemma",
    bf16=True,  # If on Ampere+ GPU
    dataloader_pin_memory=False,
    report_to=["tensorboard"],
)

# Trainer
trainer = Trainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    args=args
)

# Train
trainer.train()

# Save the fine-tuned model (adapter only if LoRA)
trainer.save_model("kaggle/working/finetuned_paligemma")



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


NameError: name 'np' is not defined

In [ ]:
# Testing before and after fine-tuning
# For before: Use original model
original_model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
original_processor = PaliGemmaProcessor.from_pretrained(model_id)  # Same processor

test_image = Image.open("path/to/your_test_image.jpg").convert("RGB")
prompt = "caption en\n"  # Or "describe this image"
inputs = original_processor(text=prompt, images=test_image, return_tensors="pt").to(device)
output_ids = original_model.generate(**inputs, max_new_tokens=50, do_sample=False)
print("Before fine-tuning:")
print(original_processor.decode(output_ids[0], skip_special_tokens=True))

# For after: Load fine-tuned (if LoRA, merge or use as is)
from peft import PeftModel
finetuned_model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
finetuned_model = PeftModel.from_pretrained(finetuned_model, "path/to/save/finetuned_paligemma")

inputs = original_processor(text=prompt, images=test_image, return_tensors="pt").to(device)
output_ids = finetuned_model.generate(**inputs, max_new_tokens=50, do_sample=False)
print("After fine-tuning:")
print(original_processor.decode(output_ids[0], skip_special_tokens=True))